# 1. Configurar ambiente

In [ ]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('/content/drive')
folder = '/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/Chatbot'

# 2. Importar verbos

In [ ]:
# Importar la lista_verbos:
with open("/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/Chatbot/verbos/lista_verbos.pickle", "rb") as f:
  lista_verbos = pickle.load(f)

# Importar el diccionario:
with open("/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/Chatbot/verbos/verbos_irregulares.pickle", "rb") as f:
  verbos_irregulares = pickle.load(f)

#3. Tratamiento de datos

In [ ]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):

  max_sim = 0
  palabra_encontrada = palabra

  for verbo in lista_verbos:
    sim = jellyfish.jaro_winkler_similarity(palabra, verbo)
    if sim > max_sim:
      max_sim = sim
      palabra_encontrada = verbo

  if max_sim < 0.93:
    return palabra
  else:
    return palabra_encontrada

In [ ]:
raiz("hola amigo")

'hola amigo'

In [ ]:
import unicodedata

def tratamiento_texto(texto):

  # Eliminar acentos
  texto = unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('utf-8')

  # Pasar a minúsculas
  texto = texto.lower()

  # Eliminar espacios en blanco extras
  texto = re.sub('\s+', ' ', texto).strip()

  return texto

In [ ]:
tratamiento_texto('¡Buen dia! ¿Cómo está todo           hoy?')

'buen dia! como esta todo hoy?'

In [ ]:
#Función para reemplazar el final de una palabra por 'r'

terminaciones = ['es', 'me', 'as', 'ste', 'te']
excluir = ['me', 'se','ser','estar','estas']
ends = ('rte', 'a')

def reemplazar_terminacion(palabra):

  if palabra in excluir:
    return palabra

  if len(palabra) < 2:
    return palabra

  if palabra.endswith('rte'):
    return palabra
  if palabra.endswith('a'):
    return palabra

  for t in terminaciones:
    if palabra.endswith(t):
      if not palabra.endswith(u'aeiouáéíóúü'):
        return palabra[:-len(t)] + 'r'

  return palabra

In [ ]:
reemplazar_terminacion( "viniste" )

'vinir'

In [ ]:
# #Función para adicionar o eliminar tokens

# variantes = {'entender': ['entiende', 'entiendes'],
#              'hablar': ['hablame'],
#              'contar': ['cuentame'],
#              'hacer': ['hago'],
#              'saber': ['sabes']}

# def revisar_tokens(texto, tokens):
#   texto=tratamiento_texto(texto)
#   if len(tokens)==0:
#     if any(name in texto for name in ['cientifico de datos', 'data scientist']):
#         tokens.append('datascientist')
#     if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
#         tokens.append('elprofealejo')
#     if any(name in texto for name in ['ciencia de datos', 'data science']):
#         tokens.append('datascience')

#   else: #tokens no está vacio, vamos a eliminar tokens irrelevantes
#     elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]

#     nuevos_tokens = []

#     for token in tokens:
#       if token not in variantes:
#         variantes[token] = [token]

#       if any(variante in texto for variante in variantes[token]):
#         elementos_a_eliminar.append(token)

#       nuevos_tokens.append(token)

#     tokens = [t for t in tokens if t not in elementos_a_eliminar]

#   return tokens

def revisar_tokens(texto, tokens):
  texto=tratamiento_texto(texto)
  if len(tokens)==0:
    if any(name in texto for name in ['cientifico de datos', 'data scientist']):
        tokens.append('datascientist')
    if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
        tokens.append('elprofealejo')
    if any(name in texto for name in ['ciencia de datos', 'data science']):
        tokens.append('datascience')
    if any(name in texto for name in ['apicultura', 'apicola']):
        tokens.append('beekeeping')
  else:
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
  return tokens


In [ ]:
tokens = []

texto = "Hola, soy un cientifico de datos que trabaja con ciencia de datos"

tokens = revisar_tokens(texto, tokens)

print(tokens)

['datascientist', 'datascience']


In [ ]:
tokens = ['hablar', 'contar', 'entender','labor']

texto = "No me hables de eso, entiendes, o cuentame algo"

tokens = revisar_tokens(texto, tokens)

print(tokens)

['hablar', 'entender', 'labor']


In [ ]:
import nltk
nltk.download('wordnet')

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
    tokens = []
    tokens = revisar_tokens(texto, tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = verbos_irregulares.get(t.text, t.lemma_)

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    lemma = reemplazar_terminacion(lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                else:
                    tokens.append(tratamiento_texto(lemma))

    # Aplicar el lematizador
    tokens = [nltk.WordNetLemmatizer().lemmatize(palabra) for palabra in tokens]

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(texto, tokens)
    tokens_str = str(tokens)
    return tokens_str

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
frase = "Estimación del costo de producción apicola promedio en Colombia"
normalizar(frase)

"['beekeeping', 'estimacion', 'costar', 'produccion', 'apicolo', 'promedio', 'colombia']"

# 4. Cargar bases de documentos

In [ ]:
import os

ruta = '/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/Chatbot/dialogos'

lista_dialogos = []
lista_dialogos_respuesta = []
lista_tipo_dialogo = []

for archivo in os.listdir(ruta):

  if archivo.endswith('.txt'):

    tipo = archivo.split('.')[0]

    with open(os.path.join(ruta, archivo)) as f:
      lineas = f.readlines()

      for i in range(0, len(lineas), 2):

        pregunta = lineas[i].strip()
        pregunta = re.sub(r"[^\w\s+\-*/]", '', pregunta)
        pregunta = tratamiento_texto(pregunta)
        lista_dialogos.append(pregunta)

        respuesta = lineas[i+1].strip()
        lista_dialogos_respuesta.append(respuesta)

        lista_tipo_dialogo.append(tipo)

In [ ]:
# Longitud lista_dialogos
longitud_dialogos = len(lista_dialogos)

# Longitud lista_dialogos_respuesta
longitud_respuestas = len(lista_dialogos_respuesta)

# Longitud lista_tipo_dialogo
longitud_tipos = len(lista_tipo_dialogo)

print(longitud_dialogos)
print(longitud_respuestas)
print(longitud_tipos)

1192
1192
1192


In [ ]:
import pandas as pd

datos = {'dialogo': lista_dialogos,
         'respuesta': lista_dialogos_respuesta,
         'tipo': lista_tipo_dialogo,
         'interseccion': pd.Series([0] * len(lista_dialogos), dtype=float),
         'jaro_winkler': pd.Series([0] * len(lista_dialogos), dtype=float),
         'probabilidad': pd.Series([0] * len(lista_dialogos), dtype=float)
         }

df_dialogo = pd.DataFrame(datos)

df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

df_dialogo.sample(5)


,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
30,gracias por tu asistencia,Siempre estoy aquí para ayudarte en todo lo qu...,Agradecimiento,0.0,0.0,0.0
1114,nos vemos te parece,"Sí, ¡nos vemos! ¡Hasta luego!",Despedida,0.0,0.0,0.0
870,tu maldita madre mamaguevo,¡No digas eso! ¡Sé más respetuoso humano!,Otros,0.0,0.0,0.0
365,por que te llamas madre,Es el nombre elegido por mi creador.,Nombre,0.0,0.0,0.0
1132,ha sido un placer charlar contigo hasta pronto,¡Igualmente! ¡Hasta pronto!,Despedida,0.0,0.0,0.0


In [ ]:
txt_folder_path = '/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/Chatbot/documentos'

#Importando bases csv
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

print(documento_csv)

El curso 'Fundamentos de Ciencia de Datos' tiene un tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso 'Análisis Exploratorio de Datos' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso 'Aprendizaje Automático en Ciencia de Datos' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 2023. El curso 'Fundamentos de Machine Learning' tiene tiempo de duración de 8 semanas, por un precio o costo de $500 USD y con fecha de inicio en 1 de julio de 2023. El curso 'Aprendizaje Supervisado y No Supervisado' tiene tiempo de duración de 6 semanas, por un precio o costo de $400 USD y con fecha de inicio en 15 de agosto de 2023. El curso 'Redes Neuronales y Deep Learning' tiene tiempo de duración de 10 semanas, por un precio o costo de $700 USD y con fecha de inicio en 5 de septiembre de 2

In [ ]:
import docx

# DOCX
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*')+ "\n"

print(documento_docx)

ElProfeAlejo es un consultor especializado en Ciencia de Datos con una amplia experiencia en diversos ámbitos, incluyendo Business Intelligence, Big Data, Data Science, Cloud Computing y Marketing Digital. 

ElProfeAlejo nació en Lima, Perú, actualmente trabaja en São Paulo, Brasil. 

Con más de 20 años de experiencia en varios países, ElProfeAlejo ha trabajado en proyectos internacionales relacionados con Business Intelligence, Big Data e Innovación Tecnológica, ha interactuado diariamente con consultores, gestores y directores de diversas áreas de Telefónica Brazil y Telefónica Global, lo que le ha permitido adquirir un amplio conocimiento y experiencia en el campo. 

ElProfeAlejo posee un conocimiento sólido de diversos lenguajes, herramientas y softwares utilizados en Ciencia de Datos, como SAS, Teradata, SQL Server, R, Python, Spark, Hive, Machine Learning, Tableau, MicroStrategy, Power BI, PowerPivot, entre otros, su capacidad analítica es predominante y se destaca por su excelen

In [ ]:
# TXT
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i

print(documento_txt)

La ciencia de datos es un campo académico interdisciplinario que utiliza estadística, computación científica, métodos, procesos, algoritmos y sistemas científicos para obtener (recolectar o extraer), tratar, analizar y presentar informes a partir de datos ruidosos, estructurados y no estructurados. 

La ciencia de datos es multifacética y puede describirse como una ciencia, un paradigma de investigación, un método de investigación, una disciplina, un flujo de trabajo o una profesión. 

La ciencia de datos integra el conocimiento del dominio de la aplicación subyacente (por ejemplo, ciencia económica, finanzas, medicina, ciencias naturales, tecnologías de la información) y es un "concepto consistente en unificar estadística, análisis de datos, informática y sus métodos relacionados" para "comprender y analizar fenómenos reales" con datos. 

La ciencia de datos utiliza métodos, técnicas y teorías extraídas de muchos campos dentro del contexto de las matemáticas, las estadísticas, las cie

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2

# Lista de documentos PDF
lista_documentos = [x for x in os.listdir(txt_folder_path) if x.endswith(".pdf")]

# Variable para guardar el contenido del documento PDF
documento_pdf = ""

# Iterar sobre la lista de documentos PDF
for documento in lista_documentos:

    # Abrir el documento PDF
    with open(os.path.join(txt_folder_path, documento), "rb") as f:
        reader = PyPDF2.PdfReader(f)

        # Extraer el texto de cada página del documento PDF
        for page in reader.pages:
            documento_pdf += page.extract_text()

print(documento_pdf)

Es  fundamental  tener  en  cuenta  que  los  costos  de  producción  para  los  productos   de  la  
colmena  deben  ser  calculados  para  cada  sistema  de  producción,  debido  a   que  dependen,  
en  su  gran  mayoría,  de  las  condiciones  ambientale s,  que  no  puede   27 Capítulo I   Apicultura 
en Colombia  controlar  el  productor.   Prólogo  La  contribución  de  las  abejas  a  la  
competitividad  del  sector   pecuario  es  inmensa,  tanto  por  los  productos  apícolas  que  se   
obtienen  de  la  colmena  como  por  el  papel  que  cumplen  estos   insectos  en  la  polinización  
de  muchos  cultivos  de  importancia   económica. 25 Capítulo I   Apicultura en Colombia  En  
nuestro  país  las  regiones  Andina  y  Atlántica  presentan  la  mayor   cantidad  de   colmenas,  por  
lo  tanto,  en  estas  zonas  se  registra  la  mayor  producción  de  miel   (Cámara  Procultivos  de  la   
andi ,  2017).   Asimismo,  esta  actividad  promueve  el  desarrollo   tecnoló

In [ ]:
documento = documento_csv + documento_txt + documento_docx + documento_pdf
lista_frases = nltk.sent_tokenize(documento,'spanish')
lista_frases_normalizadas = [' '.join(normalizar(x)) for x in lista_frases]

# 5. Buscar respuesta del Chatbot

In [ ]:
import jellyfish
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def interseccion(texto1, texto2):
  palabras1 = set(texto1.split())
  palabras2 = set(texto2.split())

  if len(palabras1) == 0:
    return 0
  else:
    interseccion = palabras1 & palabras2
    return len(interseccion) / len(palabras1)


def similarity(texto1, texto2):
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform([texto1, texto2])
  return cosine_similarity(vectors)[0][1]

def jaro_winkler(texto1, texto2):
  return jellyfish.jaro_winkler_similarity(texto1, texto2)

In [ ]:
#Función para verificar si el usuário inició un diálogo
def dialogo(user_response):
  user_response = tratamiento_texto(user_response) #Tratando el texto
  user_response = re.sub(r"[^\w\s+\-*/?¡¿!]", '', user_response) #Elimina signos de puntuación
  df = df_dialogo.copy()
  for idx,row in df.iterrows():
    df.at[idx, 'interseccion'] = interseccion(user_response, row['dialogo'])
    df.at[idx, 'similarity'] = similarity(user_response, row['dialogo'])
    df.at[idx, 'jaro_winkler'] = jaro_winkler(user_response, row['dialogo'])
    df.at[idx,'probabilidad'] = max(df.at[idx,'interseccion'],df.at[idx,'similarity'],df.at[idx,'jaro_winkler'])
  df.sort_values(by=['probabilidad','jaro_winkler'], inplace=True, ascending=False)
  probabilidad = df['probabilidad'].head(1).values[0]
  if probabilidad >= 0.92:
    print('Respuesta encontrada por el método de comparación de textos - Probabilidad: ', probabilidad)
    respuesta = df['respuesta'].head(1).values[0]
  else:
    respuesta = ''
    # print(probabilidad)
  return respuesta

In [ ]:
dialogo("la apicultura")

Respuesta encontrada por el método de comparación de textos - Probabilidad:  1.0


'es una práctica en la que las colmenas se mueven de un lugar a otro en busca de diferentes fuentes de néctar y polen.'

In [ ]:
#Cargar tu modelo entrenado aqui(recuerda siempre cargar el modelo y el vectorizer o tokenizer usado en el entrenamiento del modelo):
ruta_modelo = '/content/drive/MyDrive/BootCamp/CHALLENGE/Spring_4/modelo/transformers'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

In [ ]:
from sklearn.preprocessing import LabelEncoder

#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,bueno te elogio a ti,¡Muchas gracias! Me alegro mucho de oír eso.,Agradecimiento,0.0,0.0,0.0,"[ ' b u e n o ' , ' t u ' , ' e l o g i o ' ]",0
1,de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,[ ' a c u e r d o ' ],0
2,entiendo,¡Perfecto! ¡Me alegra que hayas entendido!,Agradecimiento,0.0,0.0,0.0,[ ' e n t e n d e r ' ],0
3,estoy de acuerdo,¡Gracias!,Agradecimiento,0.0,0.0,0.0,"[ ' e s t a r ' , ' a c u e r d o ' ]",0
4,igualmente,¡Igualmente!,Agradecimiento,0.0,0.0,0.0,[ ' i g u a l m e n t e ' ],0
...,...,...,...,...,...,...,...,...
1186,cual es el impacto de las abejas en la producc...,contribuye a la economía global y a la disponi...,Apicultura,0.0,0.0,0.0,"[ ' s e r ' , ' i m p a c t o ' , ' a b e ...",1
1187,cual es el papel de las abejas en la produccio...,al polinizar plantas con flores. Esto asegura ...,Apicultura,0.0,0.0,0.0,"[ ' s e r ' , ' p a p e l ' , ' a b e j a ...",1
1188,cual es el impacto de la apicultura en la econ...,"generar empleo, fortalecer la agricultura loca...",Apicultura,0.0,0.0,0.0,"[ ' b e e k e e p i n g ' , ' s e r ' , ' ...",1
1189,que es la danza de las abejas y como se utiliz...,es una forma de comunicación que utilizan para...,Apicultura,0.0,0.0,0.0,"[ ' s e r ' , ' d a n z a ' , ' a b e j a ...",1


In [ ]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{0: 'Agradecimiento', 2: 'Aprendizaje', 6: 'Edad', 7: 'ElProfeAlejo', 8: 'Error', 9: 'Funcion', 10: 'Identidad', 11: 'Nombre', 12: 'Origen', 14: 'Saludos', 15: 'Sentimiento', 16: 'Usuario', 13: 'Otros', 3: 'Contacto', 4: 'Continuacion', 5: 'Despedida', 1: 'Apicultura'}


In [ ]:
def clase_encontrada(frase):
  palabra = ' '.join(normalizar(frase))

  # Realizar la predicción con el modelo
  tokens = tokenizer_TF.encode_plus(
      frase,
      add_special_tokens=True,
      max_length=128,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )

  input_ids = tokens['input_ids']
  attention_mask = tokens['attention_mask']

  with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

  etiquetas_predichas = torch.argmax(outputs.logits, dim=1)
  etiquetas_decodificadas = etiquetas_predichas.tolist()

  # diccionario = {14: 'Sentimiento', 13: 'Saludos', 10: 'Nombre', 9: 'Identidad', 6: 'ElProfeAlejo', 1: 'Aprendizaje', 8: 'Funcion', 15: 'Usuario', 11: 'Origen', 5: 'Edad', 0: 'Agradecimiento', 3: 'Continuacion', 2: 'Contacto', 4: 'Despedida', 12: 'Otros', 7: 'Error'}
  llave_buscada = etiquetas_decodificadas[0]
  clase_encontrada = relacion_diccionario[llave_buscada]

  return clase_encontrada

In [ ]:
#Función para dialogar utilizando el modelo
def clasificacion_modelo(pregunta):

  clase_encontrada1 = clase_encontrada(pregunta)

  # print(clase_encontrada1)

  #Buscar respuesta más parecida en la clase encontrada
  df = df_dialogo[df_dialogo['tipo'] == clase_encontrada1]
  df.reset_index(inplace=True)
  vectorizer = TfidfVectorizer()
  dialogos_num = vectorizer.fit_transform(df['dialogo'])
  pregunta_num = vectorizer.transform([tratamiento_texto(pregunta)])
  similarity_scores = cosine_similarity(dialogos_num, pregunta_num)
  indice_pregunta_proxima = similarity_scores.argmax()

  # print(max(similarity_scores))

  if max(similarity_scores)>0.5 and clase_encontrada1 not in ['Otros']:
    print('Respuesta encontrada por el modelo Transformers - tipo:',clase_encontrada1)
    respuesta = df['respuesta'][indice_pregunta_proxima]
  else:
    respuesta = ''
  return respuesta

In [ ]:
#Función para devolver la respuesta de los documentos
def respuesta_documento(pregunta):
  pregunta = normalizar(pregunta)
  def contar_coincidencias(frase):
    return sum(1 for elemento in pregunta if elemento in frase)

  diccionario = {valor: posicion for posicion, valor in enumerate(lista_frases_normalizadas)}
  lista = sorted(list(diccionario.keys()), key=contar_coincidencias, reverse=True)[:100]
  #Hasta aqui ya tengo mi lista con las 6 respuestas con mayor coincidencia de tokens
  # Convierte la pregunta en frase
  lista.append(''.join(pregunta))

  TfidfVec = TfidfVectorizer(tokenizer=normalizar)
  tfidf = TfidfVec.fit_transform(lista)

  vals = cosine_similarity(tfidf[-1], tfidf)
  idx = vals.argsort()[0][-2]
  flat = vals.flatten()
  flat.sort()
  req_tfidf = round(flat[-2],2)
  if req_tfidf>=0.22:
    print('Respuesta encontrada por el método TfidfVectorizer - Probabilidad:', req_tfidf)
    respuesta = lista_frases[diccionario[lista[idx]]]
  else:
    respuesta = ''
  return respuesta

In [ ]:
respuesta_documento("que es la apicultura")

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Respuesta encontrada por el método TfidfVectorizer - Probabilidad: 0.23


'Estos aumentos están dados por la \nincursión de los  productos apícolas en mercados internacionales de calidad.'

# 6. Ejecutar Chatbot

In [ ]:
#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta_df_dialogo = dialogo(pregunta)
  if respuesta_df_dialogo != '':
    return respuesta_df_dialogo

  respuesta_modelo = clasificacion_modelo(pregunta)
  if respuesta_modelo != '':
    return respuesta_modelo

  respuesta_document = respuesta_documento(pregunta)
  if respuesta != '':
    return respuesta_document

  else:
    return 'Respuesta no encontrada'

In [ ]:
pregunta='Cual es el proceso para la producción de miel?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)

#7. Con GPU

In [ ]:
#Install packages
!nvidia-smi
!pip install deep-translator
!pip install -q -U bitsandbytes loralib einops
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install accelerate
from deep_translator import GoogleTranslator

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import deep_translator

def traducir_y_responder(pregunta):
  """
  Función para devolver la respuesta del modelo LLM Falcon-7b

  Args:
    pregunta: La pregunta que se quiere traducir y responder

  Returns:
    La respuesta del modelo LLM Falcon-7b
  """

  # Traducir la pregunta al inglés
  # translator = deep_translator.GoogleTranslator(source='es', target='en')
  translator = GoogleTranslator(source='es', target='en')
  pregunta_en = translator.translate(pregunta)

  # Obtener la respuesta del modelo LLM Falcon-7b
  sequences = pipeline(
    pregunta_en,
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
  )
  respuesta = ''
  for seq in sequences:
    respuesta += f"Result: {seq['generated_text']}"

  # Traducir la respuesta al español
  respuesta_en = '\n'.join(respuesta.splitlines()[1:])
  # translator = deep_translator.GoogleTranslator(source='en', target='es')
  translator = GoogleTranslator(source='en', target='es')
  respuesta = translator.translate(respuesta_en)

  return respuesta

In [ ]:
traducir_y_responder('Prefieres trabajar de dia o de noche?')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'Como modelo de lenguaje de IA, no tengo la capacidad de trabajar ni estar despierto, pero prefiero trabajar durante la noche, ya que es el período más activo y productivo para los humanos.'

In [ ]:
#Función para devolver una respuesta final buscada en todos los métodos disponibles
def respuesta_chatbot(pregunta):
  respuesta_df_dialogo = dialogo(pregunta)
  if respuesta_df_dialogo != '':
    return respuesta_df_dialogo

  respuesta_modelo = clasificacion_modelo(pregunta)
  if respuesta_modelo != '':
    return respuesta_modelo

  respuesta_document = respuesta_documento(pregunta)
  if respuesta != '':
    return respuesta_document

  respuesta_pipeline = traducir_y_responder(pregunta)
  if respuesta != '':
    return traducir_y_responder

  else:
    return 'Respuesta no encontrada'

In [ ]:
pregunta='Cual es el proceso para la producción de miel?'
respuesta = respuesta_chatbot(pregunta)
print(respuesta)